# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'02-14-2021'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'02-13-2021'

In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2021-02-14 05:23:43,33.93911,67.709953,55473,2427,48392,4654.0,Afghanistan,142.500293,4.375101
1,NaN,NaN,NaN,Albania,2021-02-14 05:23:43,41.15330,20.168300,91987,1543,55983,34461.0,Albania,3196.434777,1.677411
2,NaN,NaN,NaN,Algeria,2021-02-14 05:23:43,28.03390,1.659600,110513,2935,75816,31762.0,Algeria,252.019091,2.655796
3,NaN,NaN,NaN,Andorra,2021-02-14 05:23:43,42.50630,1.521800,10463,107,9868,488.0,Andorra,13541.707112,1.022651
4,NaN,NaN,NaN,Angola,2021-02-14 05:23:43,-11.20270,17.873900,20329,491,18790,1048.0,Angola,61.853691,2.415269


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/4/21,2/5/21,2/6/21,2/7/21,2/8/21,2/9/21,2/10/21,2/11/21,2/12/21,2/13/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,55231,55265,55330,55335,55359,55384,55402,55420,55445,55473
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,81993,83082,84212,85336,86289,87528,88671,89776,90835,91987
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,108381,108629,108629,109088,109313,109559,109782,110049,110303,110513


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/4/21,2/5/21,2/6/21,2/7/21,2/8/21,2/9/21,2/10/21,2/11/21,2/12/21,2/13/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,2407,2407,2408,2410,2413,2414,2418,2419,2424,2427
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,1418,1433,1446,1460,1472,1488,1503,1517,1531,1543
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,2904,2909,2909,2914,2918,2924,2926,2930,2932,2935


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/4/21,2/5/21,2/6/21,2/7/21,2/8/21,2/9/21,2/10/21,2/11/21,2/12/21,2/13/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,47995,48018,48027,48041,48042,48109,48133,48279,48390,48392
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,49543,50219,50875,51444,52128,52933,53616,54362,55243,55983
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,74136,74330,74330,74692,74871,75057,75243,75436,75628,75816


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,2/4/21,2/5/21,2/6/21,2/7/21,2/8/21,2/9/21,2/10/21,2/11/21,2/12/21,2/13/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,5811,5824,5856,5869,5881,5910,5930,5970,5984,6002
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,18494,18568,18668,18723,18763,18824,18888,18960,18994,19051
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,1989,1994,2002,2008,2008,2019,2024,2030,2036,2040


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,2/4/21,2/5/21,2/6/21,2/7/21,2/8/21,2/9/21,2/10/21,2/11/21,2/12/21,2/13/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,74,76,77,77,77,77,79,81,84,84
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,233,234,227,227,227,228,239,240,251,252
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,43,44,45,45,45,45,46,46,48,48


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
651,1001.0,Autauga,Alabama,US,2021-02-14 05:23:43,32.539527,-86.644082,6002,84,0,5918.0,"Autauga, Alabama, US",10742.988061,1.399533
688,1075.0,Lamar,Alabama,US,2021-02-14 05:23:43,33.779950,-88.096680,1277,32,0,1245.0,"Lamar, Alabama, US",9250.271641,2.505873
689,1077.0,Lauderdale,Alabama,US,2021-02-14 05:23:43,34.901719,-87.656247,8460,201,0,8259.0,"Lauderdale, Alabama, US",9123.359467,2.375887


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,27575344,484248,0
India,10904940,155642,10611731
Brazil,9809754,238532,8740445


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,27575344,484248,0,27091096,2021-02-14 05:23:43,37.936303,-91.379001
India,10904940,155642,10611731,137567,2021-02-14 05:23:43,23.088275,81.806127
Brazil,9809754,238532,8740445,830777,2021-02-14 05:23:43,-12.669522,-48.480493
United Kingdom,4038929,117128,10850,3910951,2021-02-14 05:23:43,30.744028,-35.367255
Russia,4012538,78403,3538241,395894,2021-02-14 05:23:43,54.546312,62.120860


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,3480228,46874,0
Texas,2560393,41149,0
Florida,1821937,28683,0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,3480228,46874,0,3433354,2021-02-14 05:23:43,37.843962,-120.728594
Texas,2560393,41149,0,2519244,2021-02-14 05:23:43,31.660643,-98.653069
Florida,1821937,28683,0,1793254,2021-02-14 05:23:43,28.940755,-82.700744
New York,1531540,45751,0,1485789,2021-02-14 05:23:43,42.544151,-75.474183
Illinois,1160523,22087,0,1138436,2021-02-14 05:23:43,39.843618,-89.178330


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,1164880,18993,0
Arizona,Maricopa,497074,8478,0
Illinois,Cook,465341,9120,0
Florida,Miami-Dade,392842,5141,0
Texas,Harris,335673,4735,0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,1164880,18993,0,1145887,2021-02-14 05:23:43,34.308284,-118.228241,6037.0
Arizona,Maricopa,497074,8478,0,488596,2021-02-14 05:23:43,33.348359,-112.491815,4013.0
Illinois,Cook,465341,9120,0,456221,2021-02-14 05:23:43,41.841448,-87.816588,17031.0
Florida,Miami-Dade,392842,5141,0,387701,2021-02-14 05:23:43,25.611236,-80.551706,12086.0
Texas,Harris,335673,4735,0,330938,2021-02-14 05:23:43,29.858649,-95.393395,48201.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-02-11,55420,89776,110049,10391,20261,381,2008345,168676,28887,429139,...,4010376,47254,79303,1,131828,2140,165535,2134,66598,34864
2021-02-12,55445,90835,110303,10427,20294,419,2015496,168830,28892,430870,...,4025574,47828,79340,1,132259,2142,166200,2136,67681,35045
2021-02-13,55473,91987,110513,10463,20329,427,2021553,169022,28898,432303,...,4038929,48459,79381,1,132743,2195,166800,2136,68454,35104


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-02-11,2419,1517,2930,106,487,9,49874,3135,909,8138,...,115748,524,622,0,1260,35,1910,616,914,1364
2021-02-12,2424,1531,2932,106,490,9,50029,3138,909,8170,...,116507,527,622,0,1267,35,1918,616,931,1393
2021-02-13,2427,1543,2935,107,491,9,50188,3140,909,8195,...,117128,533,622,0,1273,35,1930,616,940,1398


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-02-11,48279,54362,75436,9781,18710,197,1806260,160670,26112,407155,...,10598,41426,77726,1,123811,1528,154727,1428,59989,29471
2021-02-12,48390,55243,75628,9833,18786,199,1814160,160838,26116,408426,...,10655,41966,77780,1,124253,1529,155362,1430,61302,29959
2021-02-13,48392,55983,75816,9868,18790,199,1820965,160942,26120,409877,...,10850,42505,77890,1,124734,1529,155950,1430,62575,30089


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,2/4/21,2/5/21,2/6/21,2/7/21,2/8/21,2/9/21,2/10/21,2/11/21,2/12/21,2/13/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,5811,5824,5856,5869,5881,5910,5930,5970,5984,6002
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,1260,1264,1267,1269,1272,1275,1275,1275,1275,1277
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,8298,8332,8350,8366,8380,8399,8411,8432,8442,8460
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,2699,2708,2717,2723,2729,2734,2746,2755,2762,2771
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,14149,14197,14277,14337,14361,14391,14443,14491,14524,14563


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-02-11,477570,56251,0,791106,311043,3461753,409683,266499,81875,49,...,752033,2543236,358581,13249,2505,541018,326159,126420,604683,52979
2021-02-12,478667,56405,0,793532,311608,3471311,411774,267337,82263,49,...,754279,2554842,359641,13415,2524,544209,327167,126887,605785,53086
2021-02-13,479856,56405,0,795323,312562,3480228,412983,267337,82672,49,...,756071,2560393,360584,13561,2524,547424,328047,127282,606651,53086


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                      \
Admin2         Autauga Baldwin Barbour  Bibb Blount Bullock Butler Calhoun   
2021-02-11        5970   18960    2030  2377   5955    1136   1886   12539   
2021-02-12        5984   18994    2036  2385   5953    1137   1892   12577   
2021-02-13        6002   19051    2040  2393   5957    1139   1898   12629   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-02-11         3305     1738  ...    2555    563     2961      669   
2021-02-12         3313     1739  ...    2560    563     2963      669   
2021-02-13         3318     1742  ...    2560    563     2963      669   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-02-11           3586  3256  1992          0      872    617  
2021-02-12           3594  3267  1994          0      873    617  
2021-02-13           3594  3267  1994          0      873    617  

[3 rows x 3334 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-02-11,9021,280,0,14662,5199,46022,5781,7354,1262,0,...,10812,40535,1774,188,25,6958,4633,2187,6723,647
2021-02-12,9180,282,0,14834,5212,46466,5790,7381,1269,0,...,10893,40901,1785,189,25,6966,4675,2200,6734,647
2021-02-13,9242,282,0,14948,5252,46874,5814,7381,1282,0,...,10902,41149,1790,189,25,6996,4675,2201,6744,647


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2021-02-11          81     240      46   54    116      32     64     257   
2021-02-12          84     251      48   57    121      32     64     262   
2021-02-13          84     252      48   57    121      32     64     263   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-02-11           92       37  ...      25     11       23        7   
2021-02-12           94       37  ...      25     11       23        7   
2021-02-13           94       37  ...      25     11       23        7   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-02-11             34     8    12          0       26      5  
2021-02-12             34     8    12          0       26      5  
2021-02-13             34     8    12          0       26      5  

[3 rows x 3334 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-02-11,0.000325,0.012462,0.002432,0.003767,0.002524,0.088571,0.003654,0.001068,0.000277,0.003688,...,0.003388,0.010176,0.000833,0.0,0.002677,0.023434,0.004850,0.000469,0.015631,0.000000
2021-02-12,0.000451,0.011796,0.002308,0.003465,0.001629,0.099738,0.003561,0.000913,0.000173,0.004034,...,0.003790,0.012147,0.000467,0.0,0.003269,0.000935,0.004017,0.000937,0.016262,0.005192
2021-02-13,0.000505,0.012682,0.001904,0.003453,0.001725,0.019093,0.003005,0.001137,0.000208,0.003326,...,0.003318,0.013193,0.000517,0.0,0.003659,0.024743,0.003610,0.000000,0.011421,0.001684


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-02-11,0.000414,0.009315,0.001367,0.000000,0.000000,0.0,0.004026,0.001597,0.0,0.002958,...,0.005910,0.015504,0.00161,NaN,0.005587,0.0,0.000524,0.0,0.014428,0.000000
2021-02-12,0.002067,0.009229,0.000683,0.000000,0.006160,0.0,0.003108,0.000957,0.0,0.003932,...,0.006557,0.005725,0.00000,NaN,0.005556,0.0,0.004188,0.0,0.018600,0.021261
2021-02-13,0.001238,0.007838,0.001023,0.009434,0.002041,0.0,0.003178,0.000637,0.0,0.003060,...,0.005330,0.011385,0.00000,NaN,0.004736,0.0,0.006257,0.0,0.009667,0.003589


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-02-11,0.003033,0.013914,0.002565,0.005035,0.001231,0.036842,0.004527,0.001652,0.000153,0.003040,...,0.009814,0.016115,0.001005,0.0,0.003241,0.032432,0.003027,0.000000,0.018921,0.000000
2021-02-12,0.002299,0.016206,0.002545,0.005316,0.004062,0.010152,0.004374,0.001046,0.000153,0.003122,...,0.005378,0.013035,0.000695,0.0,0.003570,0.000654,0.004104,0.001401,0.021887,0.016559
2021-02-13,0.000041,0.013395,0.002486,0.003559,0.000213,0.000000,0.003751,0.000647,0.000153,0.003553,...,0.018301,0.012844,0.001414,0.0,0.003871,0.000000,0.003785,0.000000,0.020766,0.004339


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-02-11,0.003157,0.002745,NaN,0.002358,0.003559,0.003065,0.004221,0.003778,0.005082,0.0,...,0.002164,0.004811,0.003476,0.009678,0.008048,0.006884,0.004475,0.003724,0.002367,0.001986
2021-02-12,0.002297,0.002738,NaN,0.003067,0.001816,0.002761,0.005104,0.003144,0.004739,0.0,...,0.002987,0.004563,0.002956,0.012529,0.007585,0.005898,0.003091,0.003694,0.001822,0.002020
2021-02-13,0.002484,0.000000,NaN,0.002257,0.003062,0.002569,0.002936,0.000000,0.004972,0.0,...,0.002376,0.002173,0.002622,0.010883,0.000000,0.005908,0.002690,0.003113,0.001430,0.000000


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-02-11      0.006745  0.003812  0.002964  0.003801  0.003032  0.004421   
2021-02-12      0.002345  0.001793  0.002956  0.003366 -0.000336  0.000880   
2021-02-13      0.003008  0.003001  0.001965  0.003354  0.000672  0.001759   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2021-02-11      0.002125  0.003281  0.004254  0.000000  ...  0.001568   
2021-02-12      0.003181  0.003031  0.002421  0.000575  ...  0.001957   
2021-02-13      0.003171  0.004135  0.001509  0.001725  ...  0.000000   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2021-02-11      0.003565  0.001353  0.001497   0.004201  0.002155 -0.001504   
2021-02-12      0.000000  0.000675  0.000000   0.002231  0.003378  0.001004   
2021-02-13      0.000000  0.000000  0.000000   0.000000  0.000000  0.000000   

Province_State                              
Admin2         Unassigned  Washakie Weston  
2021-02-11            NaN  0.001148    0.0  
2021-02-12            NaN  0.001147    0.0  
2021-02-13            NaN  0.000000    0.0  

[3 rows x 3334 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-02-11,0.014964,0.000000,NaN,0.013829,0.004832,0.011561,0.001906,0.003822,0.013655,NaN,...,0.007548,0.009639,0.005099,0.005348,0.041667,0.003751,0.006517,0.005517,0.002685,0.0
2021-02-12,0.017626,0.007143,NaN,0.011731,0.002500,0.009648,0.001557,0.003671,0.005547,NaN,...,0.007492,0.009029,0.006201,0.005319,0.000000,0.001150,0.009065,0.005944,0.001636,0.0
2021-02-13,0.006754,0.000000,NaN,0.007685,0.007675,0.008781,0.004145,0.000000,0.010244,NaN,...,0.000826,0.006063,0.002801,0.000000,0.000000,0.004307,0.000000,0.000455,0.001485,0.0


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-02-11      0.025316  0.004184  0.000000  0.000000  0.000000  0.032258   
2021-02-12      0.037037  0.045833  0.043478  0.055556  0.043103  0.000000   
2021-02-13      0.000000  0.003984  0.000000  0.000000  0.000000  0.000000   

Province_State                                      ... Wyoming         \
Admin2         Butler   Calhoun  Chambers Cherokee  ...    Park Platte   
2021-02-11        0.0  0.000000  0.033708      0.0  ...     0.0    0.0   
2021-02-12        0.0  0.019455  0.021739      0.0  ...     0.0    0.0   
2021-02-13        0.0  0.003817  0.000000      0.0  ...     0.0    0.0   

Province_State                                                               \
Admin2         Sheridan Sublette Sweetwater Teton Uinta Unassigned Washakie   
2021-02-11          0.0      0.0        0.0   0.0   0.0        NaN      0.0   
2021-02-12          0.0      0.0        0.0   0.0   0.0        NaN      0.0   
2021-02-13          0.0      0.0        0.0   0.0   0.0        NaN      0.0   

Province_State         
Admin2         Weston  
2021-02-11        0.0  
2021-02-12        0.0  
2021-02-13        0.0  

[3 rows x 3334 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-02-11,0.000361,0.012820,0.002304,0.003712,0.002338,0.077195,0.003630,0.001012,0.000273,0.003414,...,0.003408,0.011037,0.000682,0.0,0.002993,0.017900,0.004611,0.000616,0.015766,0.001444
2021-02-12,0.000406,0.012308,0.002306,0.003588,0.001984,0.088466,0.003595,0.000962,0.000223,0.003724,...,0.003599,0.011592,0.000574,0.0,0.003131,0.009417,0.004314,0.000777,0.016014,0.003318
2021-02-13,0.000456,0.012495,0.002105,0.003520,0.001854,0.053780,0.003300,0.001050,0.000215,0.003525,...,0.003458,0.012393,0.000546,0.0,0.003395,0.017080,0.003962,0.000388,0.013718,0.002501


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-02-11,0.000787,0.009636,0.001274,0.000555,0.005585,0.049127,0.003421,0.001441,7.825361e-18,0.003767,...,0.006868,0.016749,0.000805,NaN,0.005337,6.802983e-51,0.002922,0.000407,0.016775,0.004550
2021-02-12,0.001427,0.009432,0.000978,0.000277,0.005872,0.024564,0.003264,0.001199,3.912680e-18,0.003849,...,0.006712,0.011237,0.000403,NaN,0.005446,3.401491e-51,0.003555,0.000203,0.017687,0.012906
2021-02-13,0.001332,0.008635,0.001001,0.004856,0.003957,0.012282,0.003221,0.000918,1.956340e-18,0.003455,...,0.006021,0.011311,0.000201,NaN,0.005091,1.700746e-51,0.004906,0.000102,0.013677,0.008248


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-02-11,0.001844,0.013694,0.002556,0.005670,0.002043,0.020898,0.004030,0.001568,0.000170,0.003116,...,0.008661,0.015014,0.000768,0.0,0.003426,0.017707,0.003371,0.000012,0.019456,0.004717
2021-02-12,0.002071,0.014950,0.002551,0.005493,0.003052,0.015525,0.004202,0.001307,0.000162,0.003119,...,0.007020,0.014025,0.000732,0.0,0.003498,0.009181,0.003737,0.000706,0.020672,0.010638
2021-02-13,0.001056,0.014173,0.002518,0.004526,0.001633,0.007763,0.003976,0.000977,0.000157,0.003336,...,0.012660,0.013434,0.001073,0.0,0.003684,0.004590,0.003761,0.000353,0.020719,0.007488


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-02-11,0.002993,0.003203,NaN,0.002895,0.003635,0.002994,0.003210,0.004272,0.004420,3.891021e-101,...,0.002668,0.004858,0.003254,0.007119,0.005260,0.006356,0.003788,0.003605,0.001949,0.001554
2021-02-12,0.002645,0.002970,NaN,0.002981,0.002726,0.002877,0.004157,0.003708,0.004579,1.945510e-101,...,0.002828,0.004711,0.003105,0.009824,0.006422,0.006127,0.003439,0.003649,0.001886,0.001787
2021-02-13,0.002564,0.001485,NaN,0.002619,0.002894,0.002723,0.003546,0.001854,0.004776,9.727552e-102,...,0.002602,0.003442,0.002864,0.010354,0.003211,0.006017,0.003065,0.003381,0.001658,0.000893


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-02-11      0.005197  0.003541  0.003003  0.003695  0.002337  0.002624   
2021-02-12      0.003771  0.002667  0.002979  0.003530  0.001000  0.001752   
2021-02-13      0.003390  0.002834  0.002472  0.003442  0.000836  0.001756   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2021-02-11      0.002944  0.002926  0.003578  0.000849  ...  0.000898   
2021-02-12      0.003063  0.002978  0.002999  0.000712  ...  0.001427   
2021-02-13      0.003117  0.003556  0.002254  0.001219  ...  0.000714   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2021-02-11      0.001470  0.001418  0.001311   0.003328  0.000717 -0.000090   
2021-02-12      0.000735  0.001047  0.000655   0.002779  0.002048  0.000457   
2021-02-13      0.000367  0.000523  0.000328   0.001390  0.001024  0.000229   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2021-02-11           -1.0  0.000871 -0.000062  
2021-02-12           -1.0  0.001009 -0.000031  
2021-02-13           -1.0  0.000504 -0.000016  

[3 rows x 3334 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-02-11,0.017758,0.000512,NaN,0.012447,0.005318,0.010848,0.002451,0.003795,0.007177,NaN,...,0.007851,0.008667,0.006008,0.006413,0.020833,0.005212,0.008880,0.007276,0.004027,0.004654
2021-02-12,0.017692,0.003827,NaN,0.012089,0.003909,0.010248,0.002004,0.003733,0.006362,NaN,...,0.007671,0.008848,0.006105,0.005866,0.010417,0.003181,0.008973,0.006610,0.002832,0.002327
2021-02-13,0.012223,0.001914,NaN,0.009887,0.005792,0.009514,0.003074,0.001867,0.008303,NaN,...,0.004249,0.007456,0.004453,0.002933,0.005208,0.003744,0.004486,0.003532,0.002158,0.001163


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-02-11      0.019777  0.014375  0.006283  0.005023  0.008280  0.033371   
2021-02-12      0.028407  0.030104  0.024881  0.030289  0.025692  0.016685   
2021-02-13      0.014204  0.017044  0.012440  0.015145  0.012846  0.008343   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2021-02-11      0.008195  0.010919  0.036490  0.003502  ...  0.011017   
2021-02-12      0.004097  0.015187  0.029115  0.001751  ...  0.005508   
2021-02-13      0.002049  0.009502  0.014557  0.000876  ...  0.002754   

Province_State                                                             \
Admin2                Platte  Sheridan      Sublette Sweetwater     Teton   
2021-02-11      7.695626e-07  0.005836  6.290672e-13   0.003819  0.008101   
2021-02-12      3.847813e-07  0.002918  3.145336e-13   0.001909  0.004050   
2021-02-13      1.923907e-07  0.001459  1.572668e-13   0.000955  0.002025   

Province_State                                               
Admin2                 Uinta Unassigned  Washakie    Weston  
2021-02-11      1.020505e-07  -0.501961  0.000039  0.000830  
2021-02-12      5.102523e-08  -0.501961  0.000020  0.000415  
2021-02-13      2.551261e-08  -0.501961  0.000010  0.000208  

[3 rows x 3334 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210214,AK,54282,NaN,NaN,NaN,totalTestsViral,1584548,35.0,1230.0,...,54282,0,0,1592fc2cad2c2130bf82ed57c536a31ce4e9c5ee,0,0,0,0,0,NaN
1,20210214,AL,480931,103633.0,1851608.0,NaN,totalTestsPeopleViral,2228906,1136.0,44148.0,...,2332539,0,0,963e1ca188ee7c482909c2394fb97ef96d3a5174,0,0,0,0,0,NaN
2,20210214,AR,313028,64948.0,2336418.0,NaN,totalTestsViral,2584498,669.0,14332.0,...,2649446,13,34,618dd7a4d926384f8a960d82f732b27fc6515bc1,0,0,0,0,0,NaN
3,20210214,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,...,2140,0,0,99b5122f37b2885ecf6749afeff59cb251808a19,0,0,0,0,0,NaN
4,20210214,AZ,797270,53494.0,2884387.0,NaN,totalTestsViral,7218703,2213.0,55667.0,...,3681657,30,108,023081d76e3a50286fb23ad4eceb775ce3cfe4ea,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210214,AK,54282.0,NaN,NaN,NaN,totalTestsViral,1584548.0,35.0,1230.0,...,54282,0,0,1592fc2cad2c2130bf82ed57c536a31ce4e9c5ee,0,0,0,0,0,NaN
1,20210214,AL,480931.0,103633.0,1851608.0,NaN,totalTestsPeopleViral,2228906.0,1136.0,44148.0,...,2332539,0,0,963e1ca188ee7c482909c2394fb97ef96d3a5174,0,0,0,0,0,NaN
2,20210214,AR,313028.0,64948.0,2336418.0,NaN,totalTestsViral,2584498.0,669.0,14332.0,...,2649446,13,34,618dd7a4d926384f8a960d82f732b27fc6515bc1,0,0,0,0,0,NaN
3,20210214,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,...,2140,0,0,99b5122f37b2885ecf6749afeff59cb251808a19,0,0,0,0,0,NaN
4,20210214,AZ,797270.0,53494.0,2884387.0,NaN,totalTestsViral,7218703.0,2213.0,55667.0,...,3681657,30,108,023081d76e3a50286fb23ad4eceb775ce3cfe4ea,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-02-14,AK,54282,NaN,NaN,NaN,totalTestsViral,1584548,35.0,1230.0,NaN,...,54282,0,0,1592fc2cad2c2130bf82ed57c536a31ce4e9c5ee,0,0,0,0,0,NaN
2021-02-14,AL,480931,103633.0,1851608.0,NaN,totalTestsPeopleViral,2228906,1136.0,44148.0,NaN,...,2332539,0,0,963e1ca188ee7c482909c2394fb97ef96d3a5174,0,0,0,0,0,NaN
2021-02-14,AR,313028,64948.0,2336418.0,NaN,totalTestsViral,2584498,669.0,14332.0,254.0,...,2649446,13,34,618dd7a4d926384f8a960d82f732b27fc6515bc1,0,0,0,0,0,NaN
2021-02-14,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,NaN,...,2140,0,0,99b5122f37b2885ecf6749afeff59cb251808a19,0,0,0,0,0,NaN
2021-02-14,AZ,797270,53494.0,2884387.0,NaN,totalTestsViral,7218703,2213.0,55667.0,661.0,...,3681657,30,108,023081d76e3a50286fb23ad4eceb775ce3cfe4ea,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-02-14,AK,54282.0,NaN,NaN,NaN,totalTestsViral,1584548.0,35.0,1230.0,NaN,...,54282,0,0,1592fc2cad2c2130bf82ed57c536a31ce4e9c5ee,0,0,0,0,0,NaN
2021-02-14,AL,480931.0,103633.0,1851608.0,NaN,totalTestsPeopleViral,2228906.0,1136.0,44148.0,NaN,...,2332539,0,0,963e1ca188ee7c482909c2394fb97ef96d3a5174,0,0,0,0,0,NaN
2021-02-14,AR,313028.0,64948.0,2336418.0,NaN,totalTestsViral,2584498.0,669.0,14332.0,254.0,...,2649446,13,34,618dd7a4d926384f8a960d82f732b27fc6515bc1,0,0,0,0,0,NaN
2021-02-14,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,NaN,...,2140,0,0,99b5122f37b2885ecf6749afeff59cb251808a19,0,0,0,0,0,NaN
2021-02-14,AZ,797270.0,53494.0,2884387.0,NaN,totalTestsViral,7218703.0,2213.0,55667.0,661.0,...,3681657,30,108,023081d76e3a50286fb23ad4eceb775ce3cfe4ea,0,0,0,0,0,NaN
